<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter17/Text-Image-to-Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%%capture
try: from torch_snippets import *
except:
  %pip -q install -U diffusers transformers accelerate torch-snippets lovely_tensors torchinfo pysnooper

In [3]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

...

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
pipe.enable_model_cpu_offload()

In [5]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

In [7]:
from torchinfo import summary
summary(pipe.unet, depth=4)

Layer (type:depth-idx)                                            Param #
UNet3DConditionModel                                              --
├─Conv2d: 1-1                                                     11,840
├─Timesteps: 1-2                                                  --
├─TimestepEmbedding: 1-3                                          --
│    └─Linear: 2-1                                                410,880
│    └─SiLU: 2-2                                                  --
│    └─Linear: 2-3                                                1,639,680
├─TransformerTemporalModel: 1-4                                   --
│    └─GroupNorm: 2-4                                             640
│    └─Linear: 2-5                                                164,352
│    └─ModuleList: 2-6                                            --
│    │    └─BasicTransformerBlock: 3-1                            --
│    │    │    └─LayerNorm: 4-1                                   1,024
│   

Generation code

In [10]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

prompt = 'bear running on the ocean'
negative_prompt = 'low quality'
video_duration_seconds = 3
num_frames = video_duration_seconds * 25 + 1
video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=25, num_frames=num_frames).frames

def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video_path = export_to_video(video_frames[0])
video = imageio.mimread(video_path)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

  0%|          | 0/25 [00:00<?, ?it/s]

The flow of the pipeline is discussed below. We start with the inputs - 

```python
prompt = 'A Teddy bear running in New York City'
num_frames = 31
num_inference_steps = 25
guidance_scale = 9.0
negative_prompt = 'low quality'
```

1. Get the positive and negative prompt embeddings
```python
# FROM CLIP
prompt_embeds, negative_prompt_embeds = self.encode_prompt(...)
prompt_embeds -> tensor[1, 77, 1024] f16 n=78848 (0.2Mb) x∈[-6.574, 13.023] μ=-0.169 σ=1.028 cuda:0
negative_prompt_embeds -> tensor[1, 77, 1024] f16 n=78848 (0.2Mb) x∈[-6.008, 13.023] μ=-0.169 σ=1.032 cuda:0
if do_classifier_free_guidance:
	prompt_embeds = torch.cat([negative_prompt_embeds, prompt_embeds])
```

2. Prepare the initial noise latents
```python
latents = self.prepare_latents(...)
latents -> tensor[1, 4, 31, 32, 32] f16 n=126976 (0.2Mb) x∈[-4.426, 4.203] μ=0.006 σ=0.999 cuda:0
```

3. Denoise the latents
```python

	for i, t in enumerate(timesteps):
		...
	    noise_pred = self.unet(
	        latent_model_input,
	        t,
	        encoder_hidden_states=prompt_embeds,
	        cross_attention_kwargs=cross_attention_kwargs,
	        return_dict=False,
		)
```
where self.unet is a `UNet3DConditionModel` with following inputs/outputs

```python
Module Name: UNet3DConditionModel
Input Kwargs: 
  encoder_hidden_states - tensor[2, 77, 1024] f16 n=157696 (0.3Mb) x∈[-6.574, 13.023] μ=-0.169 σ=1.030 cuda:0 - ID:#aabb11
  cross_attention_kwargs - NoneType
  return_dict - False (bool)

Input Args:
  1 - tensor[2, 4, 31, 32, 32] f16 n=253952 (0.5Mb) x∈[-4.426, 4.203] μ=0.006 σ=0.999 cuda:0 - ID:#968d2e
  2 - 999 - ID:#7ba37a

Outputs: 
  1 - tensor[2, 4, 31, 32, 32] f16 n=253952 (0.5Mb) x∈[-4.414, 4.191] μ=0.001 σ=0.996 cuda:0 - ID:#2b3647
```

4. Decode the latents into image domain and return the frames
```python
video_tensor = self.decode_latents(latents)
video = tensor2vid(video_tensor)
```

## Unet3DConditionModel
On the outset it works very much like a Unet2DConditionModel with the following key difference - 
While a `Unet2DConditionModel` works with the shape [bs, channels, h, w] (e.g., [1, 320, 32, 32]), a `Unet3DConditionModel` works with the shape [bs\*num_frames, channels, h, w] (e.g., [31, 320, 32, 32]) where both batch size and num frames are squashed into the first dimension of the tensor. Essentially it is as if `Unet3DConditionModel` is a `Unet2DConditionModel` with batch size for number of frames. Of course, this is only superficially true as we will see `Unet3DConditionModel` uses Down and Up blocks which have temporal transformers inside.

### CrossAttnDownBlock3D
Recall that in a `CrossAttnDownBlock2D` module there are resnet2d and attention2d modules being repeatedly called on the `hidden_states` tensor. The key difference in the 3D version is, that there are two additional module types called temporal convolutions and temporal attentions and the new order of computation is as follows
```python
# hidden_states -> tensor[62, 320, 32, 32]
for resnet, temp_conv, attn, temp_attn in zip(self.resnets, ..., self.temp_attns):
	hidden_states = resnet(hidden_states, temb)
	hidden_states = temp_conv(hidden_states, num_frames=num_frames)
	hidden_states = attn(
		hidden_states,
		encoder_hidden_states=encoder_hidden_states,
		cross_attention_kwargs=cross_attention_kwargs,
	)
	hidden_states = temp_attn(
		hidden_states,
		num_frames=num_frames,
		cross_attention_kwargs=cross_attention_kwargs,
	)
```
where the shape of hidden_states is preserved across all computations. 

Let's see what `temp_conv` and `temp_attn` do now.

### TemporalConvLayer
The core component of `temp_conv` is a simple `nn.Conv3d` and the forward looks like so
<img src='https://i.imgur.com/3kPRhgE.png' alt='image' height='300'>
1. The `hidden_states` tensors is first reshaped to a 5D tensor of [bs, frames, channels, height, width] and back to [bs\*frames, channels, height, width] at the end
2. There is a resnet like operation where input is directly added to the output before returning

### TransformerTemporalModel
`temp_attn` is just a normal Transformer2D model with two additional restructuring operations on the input tensor to perform self-attention on the frames dimension instead of the usual channels dimension

See the order of operations in the picture below
<img src='https://i.imgur.com/ZflJPEn.png' alt='image' height='700'>
### CrossAttnUpBlock3D and CrossAttnMidBlock3D
Since these blocks differ only in their submodules (that we already discussed above) and have no functional differences as compared to their 2D counterparts, we leave the indepth understanding of these blocks as an activity to the reader